# Dask DataFrame

Материалы: 
* Макрушин С.В. Лекция 13: Dask DataFrame
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считать данные из файлов в каталоге `accounts`. Содержат ли какие-либо из столбцов пропуски?

In [9]:
import dask
import dask.dataframe as dd

accounts = dd.read_csv('./accounts/accounts.*.csv',
                      dtype={'amount':'float64',
                            'id': 'int64',
                            'names': 'object'})
accounts

,id,names,amount
npartitions=3,,,
,int64,object,float64
,...,...,...
,...,...,...
,...,...,...


In [3]:
accounts.isna().sum().compute()

id        0
names     0
amount    3
dtype: int64

2. Подсчитать количество раз, которое то или иное имя встретилось в выборке. Вывести самое часто встречающееся имя.

In [4]:
accounts['names'].value_counts().compute()

Norbert     188147
Alice       185892
George      183249
Tim         162432
Bob         157065
Michael     148372
Ingrid      132067
Oliver      121907
Quinn       118071
Wendy       110657
Charlie     109236
Ursula      108745
Hannah      108632
Ray         108610
Sarah       104781
Victor      102656
Frank        99984
Laura        97216
Jerry        96378
Xavier       94445
Edith        89991
Zelda        89047
Kevin        84784
Dan          73293
Patricia     62881
Yvonne       61462
Name: names, dtype: int64

In [6]:
%%time
accounts.groupby('id')['amount'].sum().compute()
accounts.groupby('id')['amount'].mean().compute()

Wall time: 1.49 s


id
0        57.251407
1      1269.194462
2      -224.010391
3        24.645438
4       601.550084
          ...     
496     523.080393
497    1185.554832
498     -11.699975
499     218.070357
128     663.000000
Name: amount, Length: 500, dtype: float64

In [10]:
%%time
dask.compute(accounts.groupby('id')['amount'].sum(),
       accounts.groupby('id')['amount'].mean())

Wall time: 809 ms


(id
 0        61030.0
 1      1971059.0
 2      -732962.0
 3       149376.0
 4      3236941.0
          ...    
 496     904406.0
 497    1091896.0
 498     -93202.0
 499    1075523.0
 128        663.0
 Name: amount, Length: 500, dtype: float64,
 id
 0        57.251407
 1      1269.194462
 2      -224.010391
 3        24.645438
 4       601.550084
           ...     
 496     523.080393
 497    1185.554832
 498     -11.699975
 499     218.070357
 128     663.000000
 Name: amount, Length: 500, dtype: float64)

3. Создать новую колонку, которая является результатом от деления значения `amount` нацело на 100, если `amount` > 100, и нулём в противном случае.

## Лабораторная работа 13

__При решении данных задач не подразумевается использования других коллекций, кроме `dask.DataFrame`, если в задании явно не указано обратное.__

In [1]:
import dask.dataframe as dd
import pandas as pd
import dask.bag as db
import json
import dask

1. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Укажите, что в столбце `submitted` содержатся даты.

In [2]:
recipes = dd.read_csv('./13_dask_dataframe_data/recipes_full/recipes_full_*.csv', 
                      parse_dates=['submitted'],
                      assume_missing=True)
#recipes.head(3)

2. Выведите метаинформацию о таблице: `npartitions` и типы столбцов.

In [3]:
recipes

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
npartitions=16,,,,,,,,
,float64,object,float64,float64,datetime64[ns],float64,object,float64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [4]:
print(f" npartitions -> \n{recipes.npartitions}")
print(f" datatypes -> \n{recipes.dtypes}")

 npartitions -> 
16
 datatypes -> 
id                       float64
name                      object
minutes                  float64
contributor_id           float64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients            float64
dtype: object


3. Выведите на экран 5 первых строк таблицы. Выведите на экран 5 последних строк таблицы. В случае сообщения об ошибки объясните причину и исправьте ошибку.

In [5]:
recipes.head()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970.0,vant ivoire mickies nothing,33.0,803776.0,2019-08-22,4.0,pat and gina neely and their family own and op...,9.0
1,1089012.0,kremsils mariposa baccala cookies class borage...,23.0,51579.0,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5.0
2,1428572.0,tania lander,0.0,68884.0,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5.0
3,1400250.0,heloise milli asher doogh zojirushi,24.0,678862.0,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1.0
4,387709.0,nutty chocolate chunk cookies,47.0,489552.0,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10.0


In [6]:
recipes.tail()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
44011,1029131.0,tuti waffle snackies steakhouse,19.0,171345.0,1973-10-18,4.0,"according to a providence journal article, ama...",4.0
44012,1700703.0,noelias cheats crocante fleisch zitumbuwa,1.0,30228.0,2007-07-01,6.0,if possible sauté the onions and garlic in abo...,1.0
44013,1910650.0,rubbed restuffed pelmeni bedouin flavourful,60.0,591905.0,2009-09-26,3.0,another great recipe to add to the growing swe...,2.0
44014,713836.0,stems polpettine peezi,NaN,357389.0,2003-09-30,4.0,adapted from top secret recipes. love this!,9.0
44015,660699.0,clementines,64.0,29196.0,1973-06-03,6.0,this would make a great start to your holiday ...,8.0


4. Посчитайте, сколько строк содержит каждый из блоков (partitions).

In [7]:
for i in range(recipes.npartitions):
    print(i+1, len(recipes.partitions[i]))

1 237505
2 41450
3 235222
4 43733
5 238139
6 40816
7 234750
8 44205
9 237887
10 41068
11 234486
12 44468
13 238298
14 40656
15 234938
16 44016


recipes["id"] recipes.id recipes.loc[:,"id"]

In [8]:
#recipes.id.isna().sum().compute() #0
for i in range(recipes.npartitions):
    print(i+1, recipes.partitions[i].id.size.compute())

1 237505
2 41450
3 235222
4 43733
5 238139
6 40816
7 234750
8 44205
9 237887
10 41068
11 234486
12 44468
13 238298
14 40656
15 234938
16 44016


5. Найдите максимум в столбце `n_steps`. Визуализируйте граф вычислений для этой задачи. Прокомментируйте логику работы `dask` при решении данной задачи.

In [9]:
max_steps = recipes.n_steps.max()
max_steps.compute()

145.0

6. Посчитайте количество рецептов с группировкой по месяцам добавления отзыва в базу.

In [10]:
recipes_count = recipes.groupby(recipes['submitted'].dt.month)['id'].count().compute()
recipes_count

submitted
1     193363
2     173834
3     192389
4     186049
5     192487
6     184205
7     189337
8     187276
9     181081
10    187018
11    180974
12    183624
Name: id, dtype: int64

7. Считайте файлы из архива `reviews_full.zip` (__ЛР12__) в виде `dask.bag`. Пользуясь результатом лабораторной работы 12, рассчитайте среднее значение оценок отзывов с группировкой по месяцам. После завершения всех вычислений преобразуйте результат к `pd.Series`.

In [11]:
import re

reviews = db.read_text('./13_dask_dataframe_data/reviews_full/reviews_*.json', include_path=True)

def add_rating(review):
    res = json.loads(review[0])
    res['rating'] = int(re.findall(r'_[0-9]', review[1])[0][1])
    return res

reviews = reviews.map(add_rating)
reviews.take(1)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},)

8. Пользуясь результатами решения задач 6 и 7, создайте `pd.DataFrame`, содержащий два столбца: `mean_rating`, `recipes_count`

In [12]:
mean_rating = pd.Series(recipes_count) # просто чтобы столбец создать
result_df = pd.DataFrame({'recipes_count': recipes_count, 'mean_rating': mean_rating}).rename_axis('month')
result_df

,recipes_count,mean_rating
month,,
1,193363,193363
2,173834,173834
3,192389,192389
4,186049,186049
5,192487,192487
6,184205,184205
7,189337,189337
8,187276,187276
9,181081,181081
